In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


In [19]:
CSV_PATH = "data.xlsx - internship-data-1.csv"  
DATETIME_COL = "time"                 
FREQ = "5min"                               
VARIABLE_COLS = [
    "Cyclone_Inlet_Gas_Temp",
    "Cyclone_Material_Temp",
    "Cyclone_Outlet_Gas_draft",
    "Cyclone_cone_draft",
    "Cyclone_Gas_Outlet_Temp",
    "Cyclone_Inlet_Draft"
]
OUTPUT_DIR = "outputs"
PLOTS_DIR = "outputs/plots"
# Choose sensible thresholds (adjust with data inspection)
TEMP_SHUTDOWN_TH = 40    # Example: below 40C means shutdown
DRAFT_SHUTDOWN_TH = 20   # Example threshold for draft pressure
# Remove short flickers ("debounce" for e.g. <30 min blips)
MIN_SHUTDOWN_PERIOD = 6   # 6 x 5min = 30 min

In [7]:
def load_timeseries(csv_path=CSV_PATH, dt_col=DATETIME_COL, freq=FREQ):
    df = pd.read_csv(csv_path)
    if dt_col not in df.columns:
        raise ValueError(f"Datetime column '{dt_col}' not found. Columns: {df.columns.tolist()}")

    df[dt_col] = pd.to_datetime(df[dt_col], utc=True, errors="coerce")
    bad_dt = df[dt_col].isna().sum()
    if bad_dt > 0:
        print(f"[WARN] {bad_dt} rows have unparseable timestamps and will be dropped.")
        df = df.dropna(subset=[dt_col])

    df = df.sort_values(dt_col)
    dup_count = df.duplicated(subset=[dt_col]).sum()
    if dup_count > 0:
        print(f"[INFO] Found {dup_count} duplicate timestamps. Keeping first occurrence.")
        df = df[~df.duplicated(subset=[dt_col], keep="first")]

    df = df.set_index(dt_col)
    start, end = df.index.min(), df.index.max()
    full_index = pd.date_range(start=start, end=end, freq=freq, tz="UTC")
    df = df.reindex(full_index)

    df.index.name = dt_col
    return df

In [8]:
def summarize_gaps(df, freq=FREQ):
    missing_rows = df.isna().all(axis=1).sum()
    total_rows = len(df)
    coverage = 100 * (1 - missing_rows / total_rows)

    is_missing = df.isna().all(axis=1).astype(int)
    seg_start = (is_missing.diff().fillna(0) == 1)
    seg_end = (is_missing.diff().fillna(0) == -1)

    starts = df.index[seg_start]
    ends = df.index[seg_end]

    if len(ends) and (len(starts) and ends[0] < starts[0]):
        ends = ends[1:]
    if len(starts) > len(ends):
        ends = ends.append(pd.Index([df.index[-1]]))

    gap_segments = []
    for s, e in zip(starts, ends):
        duration = (e - s)
        gap_segments.append({"start": s, "end": e, "duration": duration})

    gaps_df = pd.DataFrame(gap_segments)

    print(f"[GAPS] Grid rows: {total_rows}, fully-missing rows: {missing_rows}, coverage: {coverage:.2f}%")
    return gaps_df

In [9]:
def missingness_report(df, variable_cols=None):
    cols = variable_cols or [c for c in df.columns]
    rep = []
    n = len(df)
    for c in cols:
        miss = df[c].isna().sum()
        rep.append({
            "column": c,
            "missing_count": int(miss),
            "missing_pct": 100 * miss / n if n else np.nan
        })
    return pd.DataFrame(rep).sort_values("missing_pct", ascending=False)

In [10]:
def duplicate_count(csv_path=CSV_PATH, dt_col=DATETIME_COL):
    df = pd.read_csv(csv_path, usecols=[dt_col])
    df[dt_col] = pd.to_datetime(df[dt_col], errors="coerce", utc=True)
    dups = df.duplicated(subset=[dt_col]).sum()
    return int(dups)

In [11]:
def outlier_ranges(df, variable_cols=None, iqr_k=1.5, mad_k=3.5):
    cols = variable_cols or [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    results = []
    for c in cols:
        series = df[c].dropna()
        if series.empty:
            results.append({
                "column": c,
                "count": 0,
                "iqr_low": np.nan,
                "iqr_high": np.nan,
                "mad_center": np.nan,
                "mad_scale": np.nan,
                "mad_low": np.nan,
                "mad_high": np.nan
            })
            continue

        q1, q3 = series.quantile([0.25, 0.75])
        iqr = q3 - q1
        iqr_low = q1 - iqr_k * iqr
        iqr_high = q3 + iqr_k * iqr

        median = series.median()
        mad = (np.abs(series - median)).median()
    
        mad_std = 1.4826 * mad if mad > 0 else np.nan
        mad_low = median - mad_k * mad_std if not np.isnan(mad_std) else np.nan
        mad_high = median + mad_k * mad_std if not np.isnan(mad_std) else np.nan
        results.append({
            "column": c,
            "count": int(series.shape[0]),
            "iqr_low": float(iqr_low),
            "iqr_high": float(iqr_high),
            "mad_center": float(median),
            "mad_scale": float(mad_std) if not np.isnan(mad_std) else np.nan,
            "mad_low": float(mad_low) if not np.isnan(mad_low) else np.nan,
            "mad_high": float(mad_high) if not np.isnan(mad_high) else np.nan,
        })
    return pd.DataFrame(results)

In [12]:
def summary_stats(df, variable_cols):
    num_cols = [c for c in variable_cols if c in df.columns]
    stats = df[num_cols].describe(percentiles=[0.05, 0.25, 0.5, 0.75, 0.95]).T
    stats.to_csv(f"{OUTPUT_DIR}/summary_statistics.csv")
    print(f"[OK] Saved summary_statistics.csv with {len(num_cols)} variables.")
    return stats

In [13]:
def correlation_matrix(df, variable_cols, method="pearson"):
    num_cols = [c for c in variable_cols if c in df.columns]
    corr = df[num_cols].corr(method=method)
    corr.to_csv(f"{OUTPUT_DIR}/correlation_matrix.csv")
    print(f"[OK] Saved correlation_matrix.csv using method='{method}'.")
    return corr

In [14]:
def pick_representative_week(df, variable_cols):
    # Heuristic: choose first continuous 7-day window with minimal missingness
    start = df.index.min()
    end = start + pd.Timedelta(days=7)
    week = df.loc[start:end, variable_cols]
    # If week has many missing, slide forward in 7-day increments to find better coverage
    best = week
    best_missing = week.isna().mean().mean()
    current_start = start
    for _ in range(156):  # check up to ~156 weeks
        s = current_start
        e = s + pd.Timedelta(days=7)
        if e > df.index.max():
            break
        candidate = df.loc[s:e, variable_cols]
        miss = candidate.isna().mean().mean()
        if miss < best_missing:
            best_missing = miss
            best = candidate
        current_start += pd.Timedelta(days=7)
    return best

In [ ]:
def pick_representative_year(df, variable_cols):
    # Use first calendar year present in index
    df_local = df.copy()
    # Handle timezone-aware vs naive
    if df_local.index.tz is not None:
        df_local = df_local.tz_convert("UTC")
    years = sorted({(ts.year) for ts in df_local.index})
    if not years:
        raise ValueError("No timestamps to select a year from.")
    y = years[0]
    year_slice = df_local.loc[str(y), variable_cols]
    return year_slice

In [ ]:
def plot_week_overlay(week_df):
    plt.figure(figsize=(14, 7))
    # Normalize for overlay visualization (z-score)
    norm = (week_df - week_df.mean()) / (week_df.std(ddof=0) + 1e-9)
    for c in norm.columns:
        plt.plot(norm.index, norm[c], label=c, linewidth=1.2)
    plt.title("Representative Week (z-scored overlay)")
    plt.xlabel("Time")
    plt.ylabel("Z-score")
    plt.legend(loc="upper right", ncol=2, fontsize=8)
    plt.tight_layout()
    out = f"{PLOTS_DIR}/week_overlay.png"
    plt.savefig(out, dpi=200)
    plt.close()
    print(f"[OK] Saved {out}")

In [17]:
def plot_year_subplots(year_df):
    cols = [c for c in year_df.columns]
    n = len(cols)
    nrows = int(np.ceil(n / 2))
    plt.figure(figsize=(16, 3.5 * nrows))
    for i, c in enumerate(cols, 1):
        ax = plt.subplot(nrows, 2, i)
        ax.plot(year_df.index, year_df[c], linewidth=0.8)
        ax.set_title(c)
        ax.set_xlabel("Time")
        ax.set_ylabel("Value")
    plt.tight_layout()
    out = f"{PLOTS_DIR}/year_subplots.png"
    plt.savefig(out, dpi=200)
    plt.close()
    print(f"[OK] Saved {out}")

In [44]:
def shutdown_analysis(df):
    # Combine shutdown conditions across variables
    shutdown = (
        (df["Cyclone_Inlet_Gas_Temp"] < TEMP_SHUTDOWN_TH) &
        (df["Cyclone_Inlet_Draft"] < DRAFT_SHUTDOWN_TH)
    )

    # Label consecutive shutdown periods
    block = (~shutdown).cumsum() if shutdown.any() else pd.Series(0, index=df.index)
    shutdown_periods = shutdown.groupby(block).transform(
        lambda x: x.sum() if x.sum() >= MIN_SHUTDOWN_PERIOD else 0
    )
    shutdown = shutdown & (shutdown_periods > 0)

    # Segment shutdown periods: Find start and end times
    df["shutdown_flag"] = shutdown.astype(int)
    changes = df["shutdown_flag"].diff().fillna(0)
    starts = df.index[changes == 1]
    ends = df.index[changes == -1]

    if len(ends) and (len(starts) and ends[0] < starts[0]):
        ends = ends[1:]
    if len(starts) > len(ends):
        ends = ends.append(pd.Index([df.index[-1]]))

    shutdown_segments = []
    for s, e in zip(starts, ends):
        shutdown_segments.append({
            "start": s,
            "end": e,
            "duration_mins": int((e - s).total_seconds() / 60)
        })

    shutdown_df = pd.DataFrame(shutdown_segments)
    shutdown_df.to_csv("outputs/shutdownperiods.csv", index=False)

    # Total downtime and event count (across all years)
    total_downtime_hrs = shutdown_df["duration_mins"].sum() / 60
    print(f"Total downtime: {total_downtime_hrs:.1f} hours")
    print(f"Number of shutdown events: {len(shutdown_df)}")
    return shutdown_df

In [45]:
def plot_shutdowns(df, shutdown_df):
    # Pick a year to visualize
    year = df.index.year.min()   # first year available
    year_df = df.loc[str(year)]

    plt.figure(figsize=(15, 6))
    plt.plot(year_df.index, year_df["Cyclone_Inlet_Gas_Temp"], label="Cyclone_Inlet_Gas_Temp", alpha=0.6)
    # Add background color for shutdown periods
    for _, row in shutdown_df[shutdown_df["start"].dt.year == year].iterrows():
        plt.axvspan(row["start"], row["end"], color="red", alpha=0.2)
    plt.title(f"Cyclone_Inlet_Gas_Temp in Year {year} (Shutdowns Highlighted)")
    plt.xlabel("Date")
    plt.ylabel("Temperature (°C)")
    plt.legend()
    plt.tight_layout()
    plt.savefig("outputs/plots/full_year_shutdowns.png", dpi=200)
    plt.close()

In [52]:
def cluster_analysis(active_df, variable_cols, n_clusters=3):
    feat_df = active_df[variable_cols]
    feat_df['InletTemp_30min_mean'] = feat_df['Cyclone_Inlet_Gas_Temp'].rolling(6).mean()
    feat_df['InletTemp_30min_std'] = feat_df['Cyclone_Inlet_Gas_Temp'].rolling(6).std()
    feat_df = feat_df.dropna()  # after rolling
    scaler = StandardScaler()
    X = scaler.fit_transform(feat_df)

    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(X)
    feat_df['cluster'] = labels
    summary = feat_df.groupby('cluster')[variable_cols].agg(['mean', 'std', 'min', 'max', 'median', lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)])
    summary.to_csv('outputs/clustersummary.csv')

    # Frequency and Duration of each state
    feat_df['period'] = (feat_df['cluster'] != feat_df['cluster'].shift()).cumsum()
    freq_table = feat_df.groupby('cluster')['period'].nunique()
    duration_table = feat_df.groupby('cluster').apply(lambda x: x.shape[0]*5/60)  # hours, as 5min per row
    print(freq_table)
    print(duration_table)

In [51]:
if __name__ == "__main__":
    # Report duplicates in raw file
    print(f"[DUPLICATES] Raw duplicate timestamps: {duplicate_count(CSV_PATH, DATETIME_COL)}")

    df = load_timeseries(CSV_PATH, DATETIME_COL, FREQ)
    for col in VARIABLE_COLS:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Gap summary and segments
    #gaps_df = summarize_gaps(df, FREQ)
    #gaps_df.to_csv("outputs/gap_segments.csv", index=False)

    # Missingness by column
    #miss_df = missingness_report(df, VARIABLE_COLS)
    #miss_df.to_csv("outputs/missingness_report.csv", index=False)

    # Outlier ranges using IQR and MAD
    #out_df = outlier_ranges(df, VARIABLE_COLS)
    #out_df.to_csv("outputs/outlier_ranges.csv", index=False)

    #print("[DONE] Wrote outputs to outputs/ folder.")
     # Compute and save stats and correlations
    #stats = summary_stats(df, VARIABLE_COLS)
    #corr = correlation_matrix(df, VARIABLE_COLS, method="pearson")

    # Select representative slices
    #week_df = pick_representative_week(df, VARIABLE_COLS)
    #year_df = pick_representative_year(df, VARIABLE_COLS)

    # Visualizations
    #plot_week_overlay(week_df)
    #plot_year_subplots(year_df)

    # Optional quick-look prints
    #print(stats.head())
    #print(corr.round(3))

    shutdown_df = shutdown_analysis(df)
    #plot_shutdowns(df, shutdown_df)
    active_df = df[df["shutdown_flag"] == 0].copy()
    cluster_analysis(active_df, VARIABLE_COLS, n_clusters=4)


[DUPLICATES] Raw duplicate timestamps: 0
Total downtime: 4418.8 hours
Number of shutdown events: 53
cluster
0    8373
1      73
2     334
3    8567
Name: period, dtype: int64
cluster
0    20252.833333
1     1496.083333
2      888.416667
3     4265.750000
dtype: float64


/var/folders/ly/1r5ypd8x4y105xpshs9ymw7h0000gn/T/ipykernel_955/1074930233.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  duration_table = feat_df.groupby('cluster').apply(lambda x: x.shape[0]*5/60)  # hours, as 5min per row
